# Importação das bibliotecas necessárias

In [1]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL

# Crie uma instância da classe AlinharETL

In [2]:
# Crie uma instância da classe
bucket = "bronze"
datamart = "FTPMicrodadosNovoCaged"
extrator_bronze = AlinharETL(bucket,datamart)

2024-09-25 19:01:04,020 - INFO - Iniciando Sessão Spark.


# Leitura da tabela

In [3]:
df_compliance_caged = extrator_bronze.criar_view_temporaria('bronze/NovoCaged/MicrodadosNovoCaged','compliance_caged')

2024-09-25 19:01:07,270 - INFO - Aguarde... Criando View (bronze/NovoCaged/MicrodadosNovoCaged)
2024-09-25 19:01:13,717 - INFO - View criada com sucesso


# Tratamentos na tabela 

In [4]:
sql_query = """
    select distinct
        competenciamov AS dsc_competencia_mov,
        regiao AS cd_regiao,
        uf AS cd_uf,
        municipio AS cd_municipio,
        secao AS cd_secao,
        subclasse AS cd_subclasse,
        saldomovimentacao AS dsc_saldo_movimentacao,
        cbo2002ocupacao AS cd_cbo_ocupacao,
        categoria AS cd_categoria,
        graudeinstrucao AS cd_grau_instrucao,
        idade AS dsc_idade,
        horascontratuais AS vl_horas_contratuais,
        racacor AS cd_raca,
        sexo AS cd_sexo,
        tipoempregador AS cd_tipo_empregador,
        tipoestabelecimento AS cd_tipo_estabelecimento,
        tipomovimentacao AS cd_tipo_movimentacao,
        tipodedeficiencia AS cd_tipo_deficiencia,
        indtrabintermitente AS cd_ind_trab_intermitente,
        indtrabparcial AS cd_ind_trab_parcial,
        salario AS vl_salario,
        tamestabjan AS cd_porte,
        indicadoraprendiz AS cd_indicador_aprendiz,
        origemdainformacao AS cd_fonte_novo_caged,
        competenciadec AS dsc_competencia_dec,
        indicadordeforadoprazo AS cd_indicador_fora_do_prazo,
        unidadesalariocodigo AS cd_unidade_salario_codigo,
        valorsalariofixo AS vl_salario_fixo
    from compliance_caged where uf = '32'
"""

In [5]:
df_compliance_caged = extrator_bronze.carregar_dados_delta(sql_query)

2024-09-25 19:01:13,731 - INFO - Aguarde... Executando Query (Delta)
2024-09-25 19:01:13,732 - INFO - 
    select 
        competenciamov AS dsc_competencia_mov,
        regiao AS cd_regiao,
        uf AS cd_uf,
        municipio AS cd_municipio,
        secao AS cd_secao,
        subclasse AS cd_subclasse,
        saldomovimentacao AS dsc_saldo_movimentacao,
        cbo2002ocupacao AS cd_cbo_ocupacao,
        categoria AS cd_categoria,
        graudeinstrucao AS cd_grau_instrucao,
        idade AS dsc_idade,
        horascontratuais AS vl_horas_contratuais,
        racacor AS cd_raca,
        sexo AS cd_sexo,
        tipoempregador AS cd_tipo_empregador,
        tipoestabelecimento AS cd_tipo_estabelecimento,
        tipomovimentacao AS cd_tipo_movimentacao,
        tipodedeficiencia AS cd_tipo_deficiencia,
        indtrabintermitente AS cd_ind_trab_intermitente,
        indtrabparcial AS cd_ind_trab_parcial,
        salario AS vl_salario,
        tamestabjan AS cd_porte,
        indi

# Gravação no datalake

In [6]:
extrator_bronze.caminho_tabela_delta = 's3a://silver/NovoCaged/MicrodadosNovoCaged'

In [7]:
extrator_bronze.salvar_delta(df_compliance_caged, 'overwrite')

2024-09-25 19:01:13,853 - INFO - Aguarde... Persistindo dados (overwrite)
2024-09-25 19:01:38,472 - INFO - Dados persistidos com sucesso
2024-09-25 19:01:38,472 - INFO - s3a://silver/NovoCaged/MicrodadosNovoCaged
2024-09-25 19:01:38,473 - INFO - Aguarde... Realizando optimize
2024-09-25 19:01:55,817 - INFO - Optimize executado com sucesso.
2024-09-25 19:01:55,825 - INFO - Aguarde... Realizando vacuum
2024-09-25 19:02:15,129 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [8]:
extrator_bronze.parar_sessao()

2024-09-25 19:02:15,671 - INFO - Sessão Spark finalizada.
